In [4]:
import pandas as pd

In [5]:
intrpr_df = pd.read_csv("../data/entry-matching-P0ADE8.tsv", sep="\t")
intrpr_df["range"] = intrpr_df.apply(lambda r: (int(r.Matches.split("..")[0]), int(r.Matches.split("..")[1])), axis=1)
intrpr_df["start pos"] = intrpr_df.apply(lambda r: r["range"][0], axis=1)
intrpr_df["end pos"] = intrpr_df.apply(lambda r: r["range"][1], axis=1)
intrpr_df

,Accession,Name,Source Database,Type,Integrated Into,Integrated Signatures,GO Terms,Protein Accession,Protein Length,Matches,range,start pos,end pos
0,G3DSA:3.30.70.1400,Aminomethyltransferase beta-barrel domains,cathgene3d,homologous_superfamily,NaN,NaN,NaN,p0ade8,326,27..113,"(27, 113)",27,113
1,IPR006222,"Aminomethyltransferase, folate-binding domain",interpro,domain,NaN,PF01571,NaN,p0ade8,326,31..135,"(31, 135)",31,135
2,IPR017703,YgfZ/GcvT conserved site,interpro,conserved_site,NaN,TIGR03317,NaN,p0ade8,326,187..251,"(187, 251)",187,251
3,IPR023758,tRNA-modifying protein YgfZ,interpro,family,NaN,MF_01175,"GO:0005542,GO:0009451",p0ade8,326,1..321,"(1, 321)",1,321
4,IPR029043,"Glycine cleavage T-protein/YgfZ, C-terminal",interpro,homologous_superfamily,NaN,SSF101790,NaN,p0ade8,326,244..325,"(244, 325)",244,325
5,IPR045179,YgfZ/GcvT,interpro,family,NaN,PTHR22602,NaN,p0ade8,326,17..299,"(17, 299)",17,299
6,MF_01175,tRNA-modifying protein YgfZ [ygfZ],hamap,family,IPR023758,NaN,NaN,p0ade8,326,1..321,"(1, 321)",1,321
7,PF01571,Aminomethyltransferase folate-binding domain,pfam,domain,IPR006222,NaN,NaN,p0ade8,326,31..135,"(31, 135)",31,135
8,PTHR22602,UNCHARACTERIZED,panther,family,IPR045179,NaN,NaN,p0ade8,326,17..299,"(17, 299)",17,299
9,SSF101790,Aminomethyltransferase beta-barrel domain,ssf,homologous_superfamily,IPR029043,NaN,NaN,p0ade8,326,244..325,"(244, 325)",244,325


In [6]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.gff import get_df_from_csv


def _update_gff_feature_description(feature, attribute):
    updated_feature = feature
    if "Note=Disordered" in attribute:
        updated_feature = "Disordered region"
    if "Note=Folate" in attribute:
        updated_feature = "Folate binding site"
    return updated_feature

gff_df = get_df_from_csv("./data/uniprot/ygfZ.gff")
gff_df = gff_df[~(gff_df["feature"].isin(["Sequence conflict", "Mutagenesis", "Beta strand", "Helix", "Turn",
#                                           "Chain"
                                         ]))]
# gff_df = gff_df[~(gff_df["attribute"].str.contains("Rho RNA-BD"))]
gff_df["feature"] = gff_df.apply(lambda r: _update_gff_feature_description(r["feature"], r["attribute"]), axis=1)
gff_df

,seqname,source,feature,start,end,score,strand,frame,attribute,9
0,P0ADE8,UniProtKB,Initiator methionine,1,1,.,.,.,Note=Removed;Ontology_term=ECO:0000269;evidenc...,NaN
1,P0ADE8,UniProtKB,Chain,2,326,.,.,.,ID=PRO_0000169834;Note=tRNA-modifying protein ...,NaN
2,P0ADE8,UniProtKB,Folate binding site,27,27,.,.,.,Note=Folate;Ontology_term=ECO:0000255;evidence...,NaN
3,P0ADE8,UniProtKB,Folate binding site,189,189,.,.,.,Note=Folate;Ontology_term=ECO:0000255;evidence...,NaN


In [7]:
def _get_barh_broken_range(r):
    return (r["start"], r["end"] - r["start"] + 1)

In [8]:
feats_df = pd.DataFrame(columns=["feature", "start", "end"])

for _, f in intrpr_df.iterrows():
    d = {"feature":f["Name"], "start":f["start pos"], "end":f["end pos"]}
    feats_df = feats_df.append(d, ignore_index=True)
    
for _, f in gff_df.iterrows():
    d = {"feature":f["feature"], "start":f["start"], "end":f["end"]}
    feats_df = feats_df.append(d, ignore_index=True)

    
# feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
# feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
feats_df

/tmp/ipykernel_22578/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_22578/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_22578/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_22578/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_22578/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed f

,feature,start,end,range
0,Aminomethyltransferase beta-barrel domains,27,113,"(27, 87)"
1,"Aminomethyltransferase, folate-binding domain",31,135,"(31, 105)"
2,YgfZ/GcvT conserved site,187,251,"(187, 65)"
3,tRNA-modifying protein YgfZ,1,321,"(1, 321)"
4,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,"(244, 82)"
5,YgfZ/GcvT,17,299,"(17, 283)"
6,tRNA-modifying protein YgfZ [ygfZ],1,321,"(1, 321)"
7,Aminomethyltransferase folate-binding domain,31,135,"(31, 105)"
8,UNCHARACTERIZED,17,299,"(17, 283)"
9,Aminomethyltransferase beta-barrel domain,244,325,"(244, 82)"


In [9]:
# interface_df = pd.read_csv('./data/mutfunc/iscR_interfaces.tab', sep='\t', header=None)
# interface_df.columns = ["pdb","prot_a","prot_b","evidence","chain","aa_wt","aa_mt","pdb_pos","uniprot_pos","mut","dG_wt","dG_mt","ddG","dG_wt_sd","dG_mt_sd", "ddG_sd"]
# # interface_df["NC_000913v3_pos"] = interface_df["uniprot_pos"] + 1  # Adding 1 to account for initiator methionine (start codon) within NC_000913v3 AA ref
# interface_df.sort_values(by="uniprot_pos", inplace=True)
# display(len(interface_df), interface_df.head())

In [10]:
# df = interface_df.copy()
# df = df.drop(columns=["aa_wt", "aa_mt", "mut", "dG_wt", "dG_mt", "ddG", "dG_wt_sd", "dG_mt_sd", "ddG_sd"]).drop_duplicates()
# df.prot_b.unique()

In [11]:
from aledbmutil.gff import group_sequences

TODO: change gene_interface_df to something more general since it's so often reused

In [12]:
# gene_interface_df = interface_df[((interface_df["prot_a"]=="P0AGK8") & (interface_df["prot_b"]=="P0AGK8"))]
# # gene_interface_df = gene_interface_df["NC_000913v3_pos"].unique()
# gene_interface_AAs = gene_interface_df["uniprot_pos"].unique()

# # Outputting ranges for manual input into R
# gene_interface_AAs.sort()
# gene_interface_AA_ranges = group_sequences(gene_interface_AAs)
# # gene_interface_AA_ranges

In [13]:
# for rng_l in gene_interface_AA_ranges:
#     feats_df = feats_df.append(
#         {
#             "feature": "IscR subunit interface",
#             "start": rng_l[0],
#             "end": rng_l[-1],
#         },
#         ignore_index=True
#     )

# display(len(feats_df), feats_df.head())

In [14]:
feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
display(len(feats_df), feats_df.head())

16

,feature,start,end,range
0,Chain,2,326,"(2, 325)"
1,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,"(244, 82)"
2,Aminomethyltransferase beta-barrel domain,244,325,"(244, 82)"
3,tRNA-modifying protein YgfZ,1,321,"(1, 321)"
4,tRNA-modifying protein YgfZ [ygfZ],1,321,"(1, 321)"


In [15]:
# The following entries are from interpro,
# https://www.ebi.ac.uk/interpro/protein/UniProt/P0ADE8/
l = [
    {'feature':'GcvT family signature motif', 'start':187, 'end':251},

    # https://www.rcsb.org/groups/sequence/polymer_entity/P0ADE8
    {'feature':'Acetate binding site', 'start':27, 'end':27},
    {'feature':'Acetate binding site', 'start':74, 'end':74},
    {'feature':'Acetate binding site', 'start':162, 'end':162},
    {'feature':'Acetate binding site', 'start':189, 'end':189},
    {'feature':'Acetate binding site', 'start':193, 'end':193},
    {'feature':'Acetate binding site', 'start':193, 'end':193},
    {'feature':'Ca binding site', 'start':286, 'end':286},
    {'feature':'Cl binding site', 'start':68, 'end':68},
    {'feature':'Cl binding site', 'start':209, 'end':210},
    {'feature':'Cl binding site', 'start':246, 'end':246},
    {'feature':'Ethandiol binding site', 'start':16, 'end':16},
    {'feature':'Ethandiol binding site', 'start':23, 'end':25},
    {'feature':'Ethandiol binding site', 'start':38, 'end':38},
    {'feature':'Ethandiol binding site', 'start':50, 'end':50},
    {'feature':'Ethandiol binding site', 'start':99, 'end':99},
    {'feature':'Ethandiol binding site', 'start':107, 'end':109},
    {'feature':'Ethandiol binding site', 'start':116, 'end':117},
    {'feature':'Ethandiol binding site', 'start':260, 'end':260},
    {'feature':'SO4 binding site', 'start':27, 'end':27},
    {'feature':'SO4 binding site', 'start':74, 'end':74},
    {'feature':'SO4 binding site', 'start':79, 'end':79},
    {'feature':'SO4 binding site', 'start':145, 'end':145},
    {'feature':'SO4 binding site', 'start':154, 'end':154},
    {'feature':'SO4 binding site', 'start':154, 'end':154},
    {'feature':'SO4 binding site', 'start':162, 'end':162},
    {'feature':'SO4 binding site', 'start':189, 'end':189},
    {'feature':'SO4 binding site', 'start':193, 'end':193},
]

df = pd.DataFrame(l)
df["range"] = df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
df = df.drop_duplicates()
feats_df = pd.concat([feats_df, df])
feats_df

,feature,start,end,range
0,Chain,2,326,"(2, 325)"
1,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,"(244, 82)"
2,Aminomethyltransferase beta-barrel domain,244,325,"(244, 82)"
3,tRNA-modifying protein YgfZ,1,321,"(1, 321)"
4,tRNA-modifying protein YgfZ [ygfZ],1,321,"(1, 321)"
5,YgfZ/GcvT,17,299,"(17, 283)"
6,UNCHARACTERIZED,17,299,"(17, 283)"
7,YgfZ/GcvT conserved site,187,251,"(187, 65)"
8,folate-binding protein YgfZ,187,251,"(187, 65)"
9,Folate-binding domain,6,242,"(6, 237)"


In [16]:
feats_df.to_pickle('./annot_df.pkl')